# Поиск слэнговых ошибок

В этом ноутбуке будет произведен поиск слэнговых ошибок для добавления кандидатов в ручном режиме. Под слэнговыми ошибками подразумеваются ошибки делаемые в неформальном общении в соцсетях. К ним не относятся слэнговые сокращения из разряда: "программа" -> "прога". 

Дело в том, что некоторые такие ошибки находятся на достаточно большом расстоянии Дамерау-Левенштейна от исходного слова, а потому их весьма сложно исправить. Некоторые из этих слов являются достаточно частыми и можно попробовать выделить и исправить наиболее частые из них.

В качестве корпуса со слэнгом возьмем данные из социальных сетей корпуса Тайга.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import re
import json
from string import punctuation
from collections import Counter
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd

import nltk
from sacremoses import MosesTokenizer, MosesDetokenizer

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

from src.models.SpellChecker import LevenshteinSearcher

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Подсчет несловарных слов

Сначала просто загрузим все данные и для каждого несловарного слова подсчитаем его частоту. В дальнейшем из этого списка можно удалить слова, которые легко исправляются (например, если есть исправление на расстоянии Дамерау-Левенштейна, равном единице).

Создадим токенизаторы.

In [5]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

Проинициализируем словарь.

In [6]:
vocab_path = os.path.join(
    DATA_PATH, 'external', 'hagen_wiktionary', 'wordforms_clear.txt'
)

with open(vocab_path, 'r') as inf:
    vocab = {
        word.strip().lower().replace('ё', 'е') for word in inf.readlines()
    }

In [7]:
print(f'Размер словаря: {len(vocab)}')

Размер словаря: 2360221


Объявим список символов, из которых может состоять слово.

In [8]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-'

In [9]:
file_path = os.path.join(DATA_PATH, 'processed', 'kenlm', 
                         'social_left_right.txt')
oov_counter = Counter()

with open(file_path, 'r') as inf:
    line_numbers = sum(1 for _ in inf)

with open(file_path, 'r') as inf:

    for sentence in tqdm(inf, total=line_numbers):
        tokenized_sentence = tokenizer(sentence)
        for word in tokenized_sentence:
            if (
                not re.fullmatch(f'[{punctuation}]+', word) 
                and re.fullmatch(f'[{alphabet}]+', word)
                and word not in vocab
            ):
                oov_counter[word] += 1

Посмотрим на размер получившегося словаря:

In [10]:
print(f'Counter size: {len(oov_counter)}')

Counter size: 394514


Выглядит весьма внушительно. Посмотрим на самые популярные слова:

In [11]:
oov_counter.most_common()[:10]

[('едро', 9181),
 ('все-таки', 6797),
 ('з', 4341),
 ('че', 4085),
 ('праймериз', 4015),
 ('чо', 3910),
 ('жирик', 3744),
 ('що', 3570),
 ('гд', 2615),
 ('щас', 2602)]

Теперь создадим соответствие между несловарными словами и их исправлениями. Было проверено чуть больше 5000 слов.

In [12]:
handcode_table = {
    'че': ['что'],
    'чо': ['что'],
    'шо': ['что'],
    'ваще': ['вообще'],
    'ща': ['сейчас'],
    'щас': ['сейчас'],
    'оч': ['очень'],
    'ничо': ['ничего'],
    'ниче': ['ничего'],
    'мож': ['может'],
    'итд': ['и т.д.'],
    'чота': ['что-то'],
    'счас': ['сейчас'],
    'шоб': ['чтобы', 'чтоб', 'что бы', 'что б'],
    'щоб': ['чтобы', 'чтоб', 'что бы', 'что б'],
    'штоле': ['что ли'],
    'тя': ['тебя'],
    'пасиб': ['спасибо'],
    'чтоль': ['что ли'],
    'седня': ['сегодня'],
    'кста': ['кстати'],
    'какбэ': ['как бы'],
    'чето': ['что-то'],
    'этож': ['это же'],
    'вапще': ['вообще'],
    'хошь': ['хочешь'],
    'чтоле': ['что ли'],
    'кароч': ['короче'],
    'какой-нить': ['какой-нибудь'],
    'моск': ['мозг'],
    'щаз': ['сейчас'],
    'шта': ['что'],
    'низзя': ['нельзя'],
    'собсно': ['собственно'],
    'канешна': ['конечно'],
    'чесслово': ['честное слово'],
    'напр': ['например'],
    'ессно': ['естественно'],
    'что-нить': ['что-нибудь'],
    'щастье': ['счастье'],
    'исчо': ['еще'],
    'кагбе': ['как бы'],
    'вааще': ['вообще'],
    'как-нить': ['как-нибудь'],
    'пасиба': ['спасибо'],
    'ишшо': ['еще'],
    'низя': ['нельзя'],
    'аффтар': ['автор'],
    'канешн': ['конечно'],
    'чойта': ['что это'],
    'какбе': ['как бы'],
    'када': ['когда'],
    'ещо': ['еще'],
    'всм': ['в смысле'],
    'где-нить': ['где-нибудь'],
    'йому': ['ему'],
    'кагбэ': ['как бы'],
    'прально': ['правильно'],
    'неск': ['несколько'],
    'штоль': ['что ли'],
    'енто': ['это'],
    'чо-то': ['что-то'],
    'всеж': ['все же'],
    'штоб': ['чтобы', 'чтоб', 'что бы', 'что б'],
    'шото': ['что-то'],
    'есличо': ['если что'],
    'нравиццо': ['нравится'],
    'соотв': ['соответственно'],
    'куда-нить': ['куда-нибудь'],
    'тыж': ['ты же'],
    'канешно': ['конечно'],
    'товарисч': ['товарищ'],
    'терь': ['теперь'],
    'щетаю': ['считаю'],
    'ищо': ['еще'],
    'карочи': ['короче'],
    'собссно': ['собственно'],
    'ни-че-го': ['ничего'],
    'штоли': ['что ли'],
    'естессно': ['естественно'],
    'врятли': ['вряд ли'],
    'собсна': ['собственно'],
    'грят': ['говорят'],
    'тч': ['т.ч.'],
    'хош': ['хочешь'],
    'ктож': ['кто же'],
    'вощем': ['в общем'],
    'весчь': ['вещь'],
    'щастья': ['счастья'],
    'пжлст': ['пожалуйста'],
    'щяс': ['сейчас'],
    'ченить': ['что-нибудь'],
    'че-нить': ['что-нибудь'],
    'чоуж': ['что уж'],
    'такшта': ['так что'],
    'кого-нить': ['кого-нибудь'],
    'хоцца': ['хочется'],
    'ващще': ['вообще'],
    'кто-нить': ['кто-нибудь'],
    'вопщем': ['в общем'],
    'чево': ['чего'],
    'щитаю': ['считаю'],
    'есчо': ['еще'],
    'какую-нить': [''],
    'ящитаю': ['я считаю'],
    'мильон': ['миллион'],
    'буит': ['будет'],
    'смари': ['смотри'],
    'штоп': ['чтобы', 'чтоб', 'что бы', 'что б'],
    'помойму': ['по-мойму'],
    'шо-то': ['что-то'],
    'низачот': ['незачет'],
    'товарисчи': ['товарищи'],
    'чейта': ['что это'],
    'пожалста': ['пожалуйста'],
    'писят': ['пятьдесят'],
    'че-т': ['что-то'],
    'хочут': ['хотят'],
    'нащот': ['насчет'],
    'ващето': ['вообще-то'],
    'иво': ['его'],
    'мущщина': ['мужчина'],
    'щастя': ['счастье', 'счастья'],
    'пасип': ['спасибо'],
    'непричем': ['ни при чем'],
    'воще': ['вообще'],
    'фтопку': ['в топку'],
    'пжалста': ['пожалуйста'],
    'немношк': ['немножко'],
    'мущина': ['мужчина'],
    'пейсатель': ['писатель'],
    'абажаю': ['обожаю'],
    'какойта': ['какой-то'],
    'аффтор': ['автор'],
    'аццкий': ['адский'],
    'такшто': ['так что'],
    'любофф': ['любовь']
}

После наблюдения за валидацией также обнаружилось, что подобные сокращения существуют и для словарных слов, добавим то, что удалось найти.

In [13]:
handcode_table.update({
    'тока': ['только'],
    'скока': ['сколько']
})

Добавим еще пару сокращений, которые известны:

In [18]:
handcode_table.update({
    'кмк': ['как мне кажется'],
    'неоч': ['не очень'],
    'кст': ['кстати']
})

Посмотрим на итоговый размер таблицы.

In [19]:
print(f'Размер таблицы: {len(handcode_table)}')

Размер таблицы: 142


Сохраним ее на диск:

In [20]:
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'w') as ouf:
    json.dump(handcode_table, ouf)

### Выводы

В этом ноутбуке нам удалось найти ошибки, которые часто совершаются пользователями соцсетей и собрать из этого таблицу с исправлениями.